In [1]:
import pandas as pd
import folium
from folium import plugins
import numpy as np


In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/sandeco/CanalSandeco/master/covid-19/brasil-io-covid-19.csv')

In [3]:
df.head()


,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-03-29,AC,Acrelândia,city,2,0.0,True,15256.0,1200013.0,13.10960,NaN
1,2020-03-29,AC,Porto Acre,city,1,0.0,True,18504.0,1200807.0,5.40424,NaN
2,2020-03-29,AC,Rio Branco,city,31,0.0,True,407319.0,1200401.0,7.61074,NaN
3,2020-03-29,AC,NaN,state,34,0.0,True,881935.0,12.0,3.85516,NaN
4,2020-03-29,AL,Importados/Indefinidos,city,4,0.0,True,NaN,NaN,NaN,NaN


# Carregando arquivo que contem latitude e longitude das cidades do Brasil.
### o codigo do ibge é a chave primária das cidades

In [4]:
cidades = pd.read_csv('https://raw.githubusercontent.com/sandeco/CanalSandeco/master/covid-19/cidades_brasil.csv')

In [5]:
cidades.head()


,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
2,5200100,Abadiânia,-16.19700,-48.7057,0,52
3,3100203,Abaeté,-19.15510,-45.4444,0,31
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15


##  Pré-processamento dos dados

In [9]:
# no arquivo original, a coluna place_type tem city e estate. Vamos deixar apenas as linhas que contenham "city"
cities = df.loc[df.place_type=="city",:]
cities.place_type.unique()


array(['city'], dtype=object)

In [10]:
cities.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-03-29,AC,Acrelândia,city,2,0.0,True,15256.0,1200013.0,13.10960,NaN
1,2020-03-29,AC,Porto Acre,city,1,0.0,True,18504.0,1200807.0,5.40424,NaN
2,2020-03-29,AC,Rio Branco,city,31,0.0,True,407319.0,1200401.0,7.61074,NaN
4,2020-03-29,AL,Importados/Indefinidos,city,4,0.0,True,NaN,NaN,NaN,NaN
5,2020-03-29,AL,Maceió,city,12,0.0,True,1018948.0,2704302.0,1.17769,NaN


In [11]:
print(len(cities))

2161


In [12]:
# definindo a chave primária para ser o codigo do ibge
cidades = cidades.set_index("codigo_ibge")

In [13]:
#temos o codigo do ibge como chave comum nas tabelas cidades e cities.
#queremos então associar a latitue e latitude de um arquivo com o outro
# na tabela cities, a chave é city_ibge_code)
cities= cities.join(cidades,on='city_ibge_code')
cities.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,nome,latitude,longitude,capital,codigo_uf
0,2020-03-29,AC,Acrelândia,city,2,0.0,True,15256.0,1200013.0,13.10960,NaN,Acrelândia,-9.82581,-66.8972,0.0,12.0
1,2020-03-29,AC,Porto Acre,city,1,0.0,True,18504.0,1200807.0,5.40424,NaN,Porto Acre,-9.58138,-67.5478,0.0,12.0
2,2020-03-29,AC,Rio Branco,city,31,0.0,True,407319.0,1200401.0,7.61074,NaN,Rio Branco,-9.97499,-67.8243,1.0,12.0
4,2020-03-29,AL,Importados/Indefinidos,city,4,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-03-29,AL,Maceió,city,12,0.0,True,1018948.0,2704302.0,1.17769,NaN,Maceió,-9.66599,-35.7350,1.0,27.0


### Vamos selecionar apenas os dados necessários para o gráfico


In [15]:
# a coluna is_last determina se é a ultima informacao de ocorrencias de casos de covid.
#então vamos selecionar apenas os campos abaixo para as linhas em que is_last=True
geo_last= cities.loc[cities.is_last==True,['city','latitude','longitude','state','confirmed','deaths']]
geo_last  #exibindo a tabela resumida

,city,latitude,longitude,state,confirmed,deaths
0,Acrelândia,-9.82581,-66.8972,AC,2,0.0
1,Porto Acre,-9.58138,-67.5478,AC,1,0.0
2,Rio Branco,-9.97499,-67.8243,AC,31,0.0
4,Importados/Indefinidos,NaN,NaN,AL,4,0.0
5,Maceió,-9.66599,-35.7350,AL,12,0.0
...,...,...,...,...,...,...
707,Taubaté,-23.01040,-45.5593,SP,1,0.0
708,Valinhos,-22.96980,-46.9974,SP,1,0.0
709,Vargem Grande Paulista,-23.59930,-47.0220,SP,2,1.0
1143,Camaquã,-30.84890,-51.8043,RS,0,NaN


In [17]:
print("Numero de cidades com caso de covid até o momento", len(geo_last))
print("Numero de casos confirmados ", geo_last.confirmed.sum())


Numero de cidades com caso de covid até o momento 330
Numero de casos confirmados  3868


In [19]:
coordenadas = geo_last[['latitude','longitude','confirmed']]
coordenadas

,latitude,longitude,confirmed
0,-9.82581,-66.8972,2
1,-9.58138,-67.5478,1
2,-9.97499,-67.8243,31
4,NaN,NaN,4
5,-9.66599,-35.7350,12
...,...,...,...
707,-23.01040,-45.5593,1
708,-22.96980,-46.9974,1
709,-23.59930,-47.0220,2
1143,-30.84890,-51.8043,0


In [20]:
# excluindo dados NaN . veja no final que os dados foram reduzidos
coordenadas = coordenadas.dropna()
coordenadas

,latitude,longitude,confirmed
0,-9.82581,-66.8972,2
1,-9.58138,-67.5478,1
2,-9.97499,-67.8243,31
5,-9.66599,-35.7350,12
6,-10.18490,-36.8376,1
...,...,...,...
706,-23.34870,-47.8461,1
707,-23.01040,-45.5593,1
708,-22.96980,-46.9974,1
709,-23.59930,-47.0220,2


In [21]:
baseMap = folium.Map(
    width="100%",
    height="100%",
    location=[-15.788497, -47.879873],
    zoom_start=4

)


In [22]:
baseMap

In [23]:
baseMap = baseMap.add_child(plugins.HeatMap(coordenadas))

In [24]:
baseMap

In [30]:
geo_last= geo_last.dropna()


In [44]:
#https://www.youtube.com/watch?v=CVSd0WKy5cs
from folium import Circle
for i in range(0,len(geo_last)):
  folium.Circle(
      location=[geo_last.iloc[i]['latitude'],geo_last.iloc[i]['longitude']],
      color = "#00ff69",
      fill='#00A1B3',
      tooltip = '<li><bold>CIDADE : ' + str(geo_last.iloc[i]['city']) +
       '<li><bold>ESTADO : ' + str(geo_last.iloc[i]['state']) +
       '<li><bold>CASOS : ' + str(geo_last.iloc[i]['confirmed']) +
       '<li><bold>ÓBITOS : ' + str(geo_last.iloc[i]['deaths']) ,
      radius = (geo_last.iloc[i]['confirmed']**1.1)

  ).add_to(baseMap)

In [45]:
baseMap
